# XGBoost (with Optuna tuning)

In [2]:
import os, re, glob
import numpy as np
import pandas as pd
import optuna
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('./data/train/train.csv').copy()

In [4]:
def feature_engineering(df, weather_df=None):
    import numpy as np
    import pandas as pd
    import holidays
    from statsmodels.tsa.seasonal import STL

    # 1) 기본 전처리
    df[['영업장명', '메뉴명']] = df['영업장명_메뉴명'].str.extract(r'^([^_ ]+)[_ ](.+)$')
    df['영업일자'] = pd.to_datetime(df['영업일자'], errors='coerce')
    df['year'] = df['영업일자'].dt.year
    df['month'] = df['영업일자'].dt.month
    df['day'] = df['영업일자'].dt.day
    df['weekday'] = df['영업일자'].dt.weekday
    df['day_of_year'] = df['영업일자'].dt.dayofyear
    df['dow_month'] = df['weekday'].astype(str) + '_' + df['month'].astype(str)

    # 2) 주기/사인코사인
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['day_sin'] = np.sin(2 * np.pi * df['day'] / 28)
    df['day_cos'] = np.cos(2 * np.pi * df['day'] / 28)
    df['weekday_sin'] = np.sin(2 * np.pi * df['weekday'] / 7)
    df['weekday_cos'] = np.cos(2 * np.pi * df['weekday'] / 7)
    df['day_of_year_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365)
    df['day_of_year_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365)

    df['weekofyear'] = df['영업일자'].dt.isocalendar().week.astype(int)
    df['week_sin'] = np.sin(2*np.pi*df['weekofyear']/52)
    df['week_cos'] = np.cos(2*np.pi*df['weekofyear']/52)

    # 3) 롤링/래그
    key = '영업장명_메뉴명'
    g = df.groupby(key)['매출수량']

    for win in [3, 7, 14, 28]:
        df[f'rolling_avg_{win}d'] = g.apply(lambda s: s.shift(1).rolling(win, min_periods=1).mean()).reset_index(level=0, drop=True)
        df[f'rolling_std_{win}d'] = g.apply(lambda s: s.shift(1).rolling(win, min_periods=2).std()).reset_index(level=0, drop=True)
        df[f'rolling_sum_{win}d'] = g.apply(lambda s: s.shift(1).rolling(win, min_periods=1).sum()).reset_index(level=0, drop=True)

    for lag in [1, 3, 7, 14]:
        df[f'sales_lag_{lag}'] = g.shift(lag)

    # === 추가 피처 3개 ===
    # 1) zscore_7
    roll_mean_7 = df['rolling_avg_7d']
    roll_std_7  = df['rolling_std_7d']
    y_shift1 = g.shift(1)
    df['zscore_7'] = ((y_shift1 - roll_mean_7) / (roll_std_7 + 1e-6)).replace([np.inf, -np.inf], 0)

    # 5) === 추가: 요청한 7일 기반 피처들만 ===
    ycol = '매출수량'

    # 5-1) EWM(7)
    df['ewm_mean_7'] = g.apply(lambda s: s.shift(1).ewm(span=7, adjust=False).mean()).reset_index(level=0, drop=True)

    # 5-2) STL(7): trend_7 / seasonal_7 / resid_7
    def _stl7(gr):
        y = gr[ycol].fillna(0).values
        if len(y) < 14:
            gr['trend_7'] = 0; gr['seasonal_7'] = 0; gr['resid_7'] = 0
            return gr
        stl = STL(y, period=7, robust=True).fit()
        gr['trend_7']   = stl.trend
        gr['seasonal_7']= stl.seasonal
        gr['resid_7']   = stl.resid
        return gr
    df = df.groupby(key, group_keys=False).apply(_stl7)

    # 5-3) 주간 로그-기울기(추세): slope_log1p_7
    def _roll_slope(a):
        n = len(a)
        if n <= 1: return 0.0
        x = np.arange(n, dtype=np.float32)
        y = a.astype(np.float32)
        sx, sy = x.sum(), y.sum()
        sxx, sxy = (x*x).sum(), (x*y).sum()
        denom = n*sxx - sx*sx
        return 0.0 if denom == 0 else (n*sxy - sx*sy)/denom

    df['slope_log1p_7'] = g.apply(
        lambda s: np.log1p(s.shift(1)).rolling(7, min_periods=2).apply(_roll_slope, raw=True)
    ).reset_index(level=0, drop=True)

    # 5-4) 분위수/강건 통계: q10/q90/IQR (7일)
    g = df.groupby('영업장명_메뉴명')['매출수량']
    q10 = g.apply(lambda s: s.shift(1).rolling(7, min_periods=1).quantile(0.10)).reset_index(level=0, drop=True)
    q90 = g.apply(lambda s: s.shift(1).rolling(7, min_periods=1).quantile(0.90)).reset_index(level=0, drop=True)
    q25 = g.apply(lambda s: s.shift(1).rolling(7, min_periods=1).quantile(0.25)).reset_index(level=0, drop=True)
    q75 = g.apply(lambda s: s.shift(1).rolling(7, min_periods=1).quantile(0.75)).reset_index(level=0, drop=True)
    
    df['roll_q10_7'] = q10.values
    df['roll_q90_7'] = q90.values
    df['roll_iqr_7'] = (q75 - q25).values

    # 4) 누적/주차/월차 통계 (원래 코드 유지)
    df['cum_sales'] = df.groupby('영업장명_메뉴명')['매출수량'].cumsum()
    df['year_week'] = df['영업일자'].dt.strftime('%Y-%U')
    df['year_month'] = df['영업일자'].dt.strftime('%Y-%m')
    df['weekly_avg_sales'] = df.groupby(['영업장명_메뉴명', 'year_week'])['매출수량'].transform('mean')
    df['weekly_std_sales'] = df.groupby(['영업장명_메뉴명', 'year_week'])['매출수량'].transform('std')
    df['weekly_sum_sales'] = df.groupby(['영업장명_메뉴명', 'year_week'])['매출수량'].transform('sum')
    df['weekly_min_sales'] = df.groupby(['영업장명_메뉴명', 'year_week'])['매출수량'].transform('min')
    df['weekly_max_sales'] = df.groupby(['영업장명_메뉴명', 'year_week'])['매출수량'].transform('max')
    df['weekly_avg_shift1'] = df.groupby('영업장명_메뉴명')['weekly_avg_sales'].shift(1)
    df['weekday_avg_sales'] = df.groupby(['메뉴명', 'weekday'])['매출수량'].transform('mean')\
    
    # 1) 하락 모멘텀
    mean7  = g.transform(lambda s: s.shift(1).rolling(7,  min_periods=1).mean())
    y_prev1 = g.shift(1)
    df['ratio_l1_to_mean7'] = (y_prev1 / (mean7 + 1e-6)).replace([np.inf, -np.inf], 0)
    df['down_momentum_7']   = (1.0 - df['ratio_l1_to_mean7']).clip(lower=0)

    
    # 결측치 처리
    num_cols = df.select_dtypes(include=['number']).columns
    fill_cols = [c for c in num_cols if c != '매출수량']
    for col in fill_cols:
        df[col] = df.groupby(key)[col].transform(lambda x: x.fillna(x.median()))
        df[col] = df.groupby(key)[col].transform(lambda x: x.ffill())
        df[col] = df[col].fillna(0)

    return df

In [5]:
train = feature_engineering(train)

In [6]:
for i in range(10):  # 0부터 9까지
    filename = f"./data/test/TEST_0{i}.csv"
    df = pd.read_csv(filename).copy()
    
    # feature_engineering 함수 적용
    df = feature_engineering(df)
    
    # 처리한 데이터 저장
    df.to_csv(f"./data/test_pre/TEST_0{i}.csv", index=False, encoding='utf-8-sig')


KeyboardInterrupt



In [ ]:
# -----------------------------
# Metric & Local Validation
# -----------------------------
DEFAULT_STORE_WEIGHTS = {'담하': 1, '미라시아': 1}

def _smape(a, p, eps=1e-12):
    return 2.0 * np.abs(a - p) / (np.abs(a) + np.abs(p) + eps)

def weighted_smape_df(df: pd.DataFrame,
                      pred_col: str,
                      actual_col: str,
                      store_col: str = '영업장명',
                      item_col: str = '영업장명_메뉴명',
                      store_weights: dict | None = None) -> float:
    if actual_col not in df.columns or pred_col not in df.columns:
        return np.nan
    valid = df[df[actual_col] != 0].copy()
    if valid.empty:
        return np.nan
    valid['_smape'] = _smape(valid[actual_col].values, valid[pred_col].values)
    total = 0.0
    wsum  = 0.0
    sw = store_weights or {}
    for store, g in valid.groupby(store_col):
        w = sw.get(store, 1.0)
        item_means = g.groupby(item_col)['_smape'].mean()
        if len(item_means) == 0:
            continue
        total += w * item_means.mean()
        wsum  += w
    return (total / wsum) if wsum > 0 else np.nan

In [ ]:
def local_validate_on_test(df_test: pd.DataFrame,
                           model,
                           features: list,
                           target_col: str = '매출수량',
                           menu_col: str = '영업장명_메뉴명',
                           store_col: str = '영업장명',
                           date_col: str = '영업일자',
                           store_weights: dict | None = None) -> float:

    # 0) FE + 인코딩
    df = feature_engineering(df_test.copy())
    df = apply_cat_maps(df, cat_maps)   # ← 추가!

    if date_col in df.columns:
        df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
        df = df.sort_values([menu_col, date_col])

    # 1) 타깃 생성
    target_cols = [f'{target_col}_tplus_{i}' for i in range(1, 8)]
    for i in range(1, 8):
        df[f'{target_col}_tplus_{i}'] = df.groupby(menu_col)[target_col].shift(-i)

    valid = df.dropna(subset=target_cols).copy()
    if valid.empty:
        return np.nan

    # 2) object 컬럼 제거된 features로 제한
    features_enc = [c for c in features if c in valid.columns and valid[c].dtype != 'O']

    missing = [c for c in features_enc if c not in valid.columns]
    if missing:
        return np.nan

    Xv = valid[features_enc].values
    yhat = model.predict(Xv)  # (n,7)

    for i in range(1, 8):
        valid[f'pred_tplus_{i}'] = yhat[:, i-1]

    scores = []
    for i in range(1, 8):
        s = weighted_smape_df(
            valid,
            pred_col=f'pred_tplus_{i}',
            actual_col=f'{target_col}_tplus_{i}',
            store_col=store_col,
            item_col=menu_col,
            store_weights=store_weights
        )
        scores.append(s)
    return float(np.nanmean(scores))

In [ ]:
def evaluate_on_valid_files(model, features, valid_files, store_col='영업장명', menu_col='영업장명_메뉴명',
                            date_col='영업일자', target_col='매출수량', store_weights=None):
    scores = []
    for path in valid_files:
        df_te = pd.read_csv(path)
        use_store_col = store_col if store_col in df_te.columns else menu_col
        s = local_validate_on_test(
            df_test=df_te,
            model=model,
            features=features,
            target_col=target_col,
            menu_col=menu_col,
            store_col=use_store_col,
            date_col=date_col,
            store_weights=store_weights
        )
        if not np.isnan(s):
            scores.append(s)
    return float(np.mean(scores)) if scores else np.inf

In [ ]:
# 경로/상수
TEST_DIR   = './data/test_pre'
TEST_GLOB  = 'TEST_*.csv'
SAMPLE_SUB = './data/sample_submission.csv'
DATE_COL   = '영업일자'
TARGET_COL = '매출수량'
MENU_COL   = '영업장명_메뉴명'
STORE_COL  = '영업장명'

train[DATE_COL] = pd.to_datetime(train[DATE_COL], errors='coerce')

# feature/target 구성
features = [c for c in train.columns if c not in [TARGET_COL, DATE_COL]]
target_cols = [f'{TARGET_COL}_tplus_{i}' for i in range(1, 8)]
if not all(c in train.columns for c in target_cols):
    train = train.sort_values([MENU_COL, DATE_COL]).copy()
    for i in range(1, 8):
        train[f'{TARGET_COL}_tplus_{i}'] = train.groupby(MENU_COL)[TARGET_COL].shift(-i)
train = train.dropna(subset=target_cols).reset_index(drop=True)
X_train = train[features].values
Y_train = train[target_cols].values

# 튜닝 검증 세트(파일)
import glob, os, re
all_test_files = sorted(glob.glob(os.path.join(TEST_DIR, TEST_GLOB)))
valid_files = all_test_files[-2:] if len(all_test_files) >= 2 else all_test_files
print('[INFO] valid files for tuning:', [os.path.basename(p) for p in valid_files])

## Optuna Objective (XGB)

In [ ]:
def make_objective_xgb(X_train, Y_train, features, valid_files):
    def objective(trial):
        lr = trial.suggest_float('learning_rate', 0.008, 0.25, log=True)
        params = {
            'n_estimators'     : trial.suggest_int('n_estimators', 300, 1600, step=100),
            'learning_rate'    : lr,
            'max_depth'        : trial.suggest_int('max_depth', 3, 12),
            'min_child_weight' : trial.suggest_float('min_child_weight', 0.5, 15.0),
            'gamma'            : trial.suggest_float('gamma', 0.0, 5.0),            # ★ 추가
            'subsample'        : trial.suggest_float('subsample', 0.55, 1.0),
            'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.55, 1.0),
            'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0), # ★ 추가
            'colsample_bynode' : trial.suggest_float('colsample_bynode', 0.6, 1.0),  # ★ 추가
            'reg_alpha'        : trial.suggest_float('reg_alpha', 1e-4, 20.0, log=True),
            'reg_lambda'       : trial.suggest_float('reg_lambda', 1e-4, 20.0, log=True),
            'max_bin'          : trial.suggest_int('max_bin', 256, 512),             # ★ 추가
            'n_jobs'           : 1,
            'random_state'     : 42,
        }
        base = xgb.XGBRegressor(
            tree_method='hist',
            objective='reg:squarederror',
            **params
        )
        model = MultiOutputRegressor(base)
        model.fit(X_train, Y_train)
        # 점포 가중치도 함께 탐색
        w_damha   = trial.suggest_float('w_담하', 1.0, 10.0)
        w_mirasia = trial.suggest_float('w_미라시아', 1.0, 10.0)
        store_weights = {'담하': w_damha, '미라시아': w_mirasia}
        score = evaluate_on_valid_files(model, features, valid_files, store_col=STORE_COL,
                                        menu_col=MENU_COL, date_col=DATE_COL, target_col=TARGET_COL,
                                        store_weights=store_weights)
        return score
    return objective

In [ ]:
# === 카테고리 인코딩 (XGB/LGBM용) ===
# (튜닝/학습 셀 위에 두세요)

# 1) 카테고리 후보 (존재하는 것만 사용)
cand_cat = ['영업장명_메뉴명','영업장명','메뉴명','dow_month','year_week','year_month']
cat_cols = [c for c in cand_cat if c in train.columns]
# object dtype 컬럼도 추가 (중복 제거)
cat_cols += [c for c in train.select_dtypes('object').columns if c not in cat_cols]
cat_cols = list(dict.fromkeys(cat_cols))

print('[INFO] categorical columns:', cat_cols)

# 2) (train + 모든 test_pre) 값의 합집합으로 카테고리 맵 생성 → unknown은 -1 처리
import glob, os

def build_cat_maps(train_df, test_dir=TEST_DIR, pattern=TEST_GLOB):
    values = {c:set(train_df[c].dropna().astype(str).unique()) for c in cat_cols}
    for path in glob.glob(os.path.join(test_dir, pattern)):
        te = pd.read_csv(path)
        te = feature_engineering(te)
        for c in cat_cols:
            if c in te.columns:
                values[c].update(te[c].dropna().astype(str).unique())
    # 안정적인 정렬(옵션): 정렬/미정렬 아무거나 사용 가능
    maps = {c:{v:i for i, v in enumerate(sorted(values[c]))} for c in cat_cols}
    return maps

cat_maps = build_cat_maps(train)

def apply_cat_maps(df, maps):
    df = df.copy()
    for c, m in maps.items():
        if c in df.columns:
            df[c] = df[c].astype(str).map(m).fillna(-1).astype(int)
    return df

# 3) 학습 데이터 인코딩 후 X/Y 재생성
train_enc = apply_cat_maps(train, cat_maps)

# 타깃 파생 접두어
forbidden_prefix = f'{TARGET_COL}_tplus_'

# 인코딩된 train_enc 기준으로 features 재구성
features = [
    c for c in train_enc.columns
    if (c not in [TARGET_COL, DATE_COL])      # 원 타깃/날짜 제외
    and (not c.startswith(forbidden_prefix))  # 매출수량_tplus_* 제외
    and (train_enc[c].dtype != 'O')            # object 제외
]

# 학습 데이터 다시 만들기
X_train = train_enc[features].values
target_cols = [f'{TARGET_COL}_tplus_{i}' for i in range(1, 8)]
Y_train = train_enc[target_cols].values

print('[INFO] len(features)=', len(features))
assert not any(c.startswith(forbidden_prefix) for c in features)

## Run Study

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(make_objective_xgb(X_train, Y_train, features, valid_files), n_trials=100, show_progress_bar=True)
print('[BEST] params:', study.best_params)
print('[BEST] score :', study.best_value)

best_params = study.best_params.copy()
best_store_weights = {'담하': best_params.pop('w_담하'), '미라시아': best_params.pop('w_미라시아')}
print('[INFO] final store weights:', best_store_weights)

In [ ]:
final_model = MultiOutputRegressor(xgb.XGBRegressor(
    tree_method='hist',
    objective='reg:squarederror',
    **best_params
))
final_model.fit(X_train, Y_train)

## Predict & Build Submission

In [ ]:
all_rows = []
for path in all_test_files:
    df_te = pd.read_csv(path)

    # 1) FE
    df_te = feature_engineering(df_te)
    # 원본 메뉴명 보존
    df_te['menu_orig'] = df_te[MENU_COL].astype(str)

    # 2) 인코딩
    df_te_enc = apply_cat_maps(df_te, cat_maps)
    # 인코딩 후에도 원본 메뉴명 열 유지
    df_te_enc['menu_orig'] = df_te['menu_orig'].values

    # 3) 피처 보강/숫자화
    for col in features:
        if col not in df_te_enc.columns:
            df_te_enc[col] = 0
    df_te_enc[features] = df_te_enc[features].apply(pd.to_numeric, errors='coerce').fillna(0).astype('float32')

    # 4) 날짜 처리
    if DATE_COL in df_te_enc.columns:
        df_te_enc[DATE_COL] = pd.to_datetime(df_te_enc[DATE_COL], errors='coerce')

    fname = os.path.basename(path)
    m = re.search(r'(TEST_\d+)', fname); test_prefix = m.group(1) if m else 'TEST_??'

    # ▶ 그룹바이 기준을 'menu_orig'(문자열)로!
    for menu_name, sub in df_te_enc.groupby('menu_orig'):
        sub = sub.sort_values(DATE_COL) if DATE_COL in sub.columns else sub
        X_last = sub[features].tail(1).to_numpy(dtype=np.float32, copy=False)
        yhat = final_model.predict(X_last).ravel()
        yhat = np.clip(yhat, 0, None)

        for k in range(7):
            all_rows.append({
                '영업일자': f'{test_prefix}+{k+1}일',
                '영업장명_메뉴명': menu_name,   # ← 문자열 이름으로 저장
                '매출수량': float(yhat[k])
            })

full_pred_df = pd.DataFrame(all_rows, columns=['영업일자','영업장명_메뉴명','매출수량'])

In [ ]:
full_pred_df

In [ ]:
full_pred_df.to_csv('./data/xgb7.csv', index=False, encoding='utf-8-sig')

# Submission

In [ ]:
def convert_to_submission_format(pred_df: pd.DataFrame, sample_submission: pd.DataFrame):
    pred_dict = dict(zip(
        zip(pred_df['영업일자'], pred_df['영업장명_메뉴명']),
        pred_df['매출수량']
    ))
    final_df = sample_submission.copy()
    for row_idx in final_df.index:
        date = final_df.loc[row_idx, '영업일자']
        for col in final_df.columns[1:]:
            final_df.loc[row_idx, col] = pred_dict.get((date, col), 0)
    return final_df

In [ ]:
sample_submission = pd.read_csv(SAMPLE_SUB)
submission = convert_to_submission_format(full_pred_df, sample_submission)
submission.head()

In [ ]:
submission.to_csv('Gonjiam_submission_XGB7.csv', index=False, encoding='utf-8-sig')